In [194]:
import sys
import numpy as np
import pandas as pd
import os
import json
from functools import reduce
from datetime import datetime
from pandas.io.json import json_normalize
from itertools import chain, starmap

## Extract and Clean Crime Data

In [195]:
# Define directories
CRIME_RECORDS_DIR = '../crime_data'
CRIME_DATA_DF_LIST = []
WEATHER_DATA_DIR = '../weather_data'

In [196]:
# Open each file, add the crime records to a list: CRIME_DATA_DF_LIST
def preprocess_file(file):
    with open("{}/{}".format(CRIME_RECORDS_DIR,file), 'r') as f:
        data = f.read()
    json_data = json.loads(json.loads(data))['dataEvents'] #returns list of crime records
    if(len(json_data) > 0):  
        tmp_df = pd.DataFrame(json_data)
        CRIME_DATA_DF_LIST.append(tmp_df)

In [197]:
# Iterate through all files in crime directory
for file in os.listdir(CRIME_RECORDS_DIR):
    if(file.endswith(".json")):
        # Preprocess file
        preprocess_file(file)

In [198]:
crime_df = pd.concat(CRIME_DATA_DF_LIST)

In [199]:
crime_df

,view34,view35,view36,view62,view81,view82,view84,view85,view86,view96,view174,view175,view177,view182
0,13-010407,Larceny,Residential,"Mar 28, 2013 3:33 PM",-73.6848,42.7360,Troy Police Department,Address,<img height='16px' width='16px' src='http://co...,22XX OLD 6TH AVE,0,496,0,08/31/2015
1,13-010265,Burglary,Residential,"Mar 27, 2013 3:00 PM",-73.6913,42.7248,Troy Police Department,Address,<img height='16px' width='16px' src='http://co...,1XX 3RD ST,0,496,0,08/31/2015
2,13-010245,Larceny,Residential,"Mar 27, 2013 12:10 PM",-73.6846,42.7125,Troy Police Department,Address,<img height='16px' width='16px' src='http://co...,1XX DELAWARE AVE,0,496,0,08/31/2015
3,13-010143,Burglary,Residential,"Mar 26, 2013 3:30 PM",-73.6913,42.7250,Troy Police Department,Address,<img height='16px' width='16px' src='http://co...,1XX 3RD ST,0,496,0,08/31/2015
4,13-010311,Larceny,School,"Mar 26, 2013 9:50 AM",-73.6706,42.7291,Troy Police Department,Address,<img height='16px' width='16px' src='http://co...,19XX BURDETT AVE,0,496,0,08/31/2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,18-005479,Agg. Assault,Street,"Jul 10, 2018 4:00 AM",-73.6887,42.7210,Troy Police Department,Intersection,<img height='16px' width='16px' src='http://co...,6TH ST & JEFFERSON AVE,0,496,0,07/23/2018
41,18-005357,Robbery,Other Outside,"Jul 04, 2018 10:16 PM",-73.6904,42.7297,Troy Police Department,Address,<img height='16px' width='16px' src='http://co...,XX 3RD ST,0,496,0,07/17/2018
42,18-005356,Larceny,Residential,"Jul 04, 2018 1:45 PM",-73.6697,42.7248,Troy Police Department,Address,<img height='16px' width='16px' src='http://co...,16XX TIBBITS AVE,0,496,0,07/17/2018
43,18-005354,Larceny,Street,"Jul 04, 2018 1:23 PM",-73.6942,42.7249,Troy Police Department,Address,<img height='16px' width='16px' src='http://co...,1XX 1ST ST,0,496,0,07/17/2018


In [193]:
column_mapping = {"view34":"Incident_ID", "view35":"Crime","view36":"Location","view62":"Date_Time","view84":"Agency","view81":"Longitude","view82":"Latitude","view85":"Accuracy","view96":"Address"}
crime_df.rename(columns=column_mapping,inplace=True)
crime_df.drop(columns=['view86','view174','view177','view182','view175'],inplace=True)

In [125]:
# # UNIT TESTING: DateTime to Unix Timestamp
# # Source: https://docs.python.org/2/library/datetime.html#strftime-and-strptime-behavior
# s = "Mar 28, 2013 3:33 PM"
# d = datetime.strptime(s, '%b %d, %Y %I:%M %p').strftime('%Y-%m-%d')
# d = datetime.strptime(d, '%Y-%m-%d')
# print(d)
# print(int(d.timestamp()))

2013-03-28 00:00:00
1364443200


In [134]:
def convert_date_to_unix(x):
    d = datetime.strptime(x, '%b %d, %Y %I:%M %p').strftime('%Y-%m-%d-%H')
    d = datetime.strptime(d, '%Y-%m-%d-%H').timestamp()
    return int(d)

In [135]:
# Add extra date column (where date is a Unix Timestamp)
crime_df['Date_Time_Unix'] = crime_df['Date_Time']
crime_df['Date_Time_Unix'] = crime_df['Date_Time_Unix'].apply(lambda x: convert_date_to_unix(x))

In [179]:
crime_df

,Incident_ID,Crime,Location,Date_Time,Longitude,Latitude,Agency,Accuracy,Address,Date_Time_Unix
0,13-010407,Larceny,Residential,"Mar 28, 2013 3:33 PM",-73.6848,42.7360,Troy Police Department,Address,22XX OLD 6TH AVE,1364497200
1,13-010265,Burglary,Residential,"Mar 27, 2013 3:00 PM",-73.6913,42.7248,Troy Police Department,Address,1XX 3RD ST,1364410800
2,13-010245,Larceny,Residential,"Mar 27, 2013 12:10 PM",-73.6846,42.7125,Troy Police Department,Address,1XX DELAWARE AVE,1364400000
3,13-010143,Burglary,Residential,"Mar 26, 2013 3:30 PM",-73.6913,42.7250,Troy Police Department,Address,1XX 3RD ST,1364324400
4,13-010311,Larceny,School,"Mar 26, 2013 9:50 AM",-73.6706,42.7291,Troy Police Department,Address,19XX BURDETT AVE,1364302800
...,...,...,...,...,...,...,...,...,...,...
40,18-005479,Agg. Assault,Street,"Jul 10, 2018 4:00 AM",-73.6887,42.7210,Troy Police Department,Intersection,6TH ST & JEFFERSON AVE,1531209600
41,18-005357,Robbery,Other Outside,"Jul 04, 2018 10:16 PM",-73.6904,42.7297,Troy Police Department,Address,XX 3RD ST,1530756000
42,18-005356,Larceny,Residential,"Jul 04, 2018 1:45 PM",-73.6697,42.7248,Troy Police Department,Address,16XX TIBBITS AVE,1530723600
43,18-005354,Larceny,Street,"Jul 04, 2018 1:23 PM",-73.6942,42.7249,Troy Police Department,Address,1XX 1ST ST,1530723600


## Extract and Clean Weather Data

In [137]:
# Source https://towardsdatascience.com/flattening-json-objects-in-python-f5343c794b10
def flatten_json(y):
    out = {}
    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x
    flatten(y)
    return out

In [138]:
def preprocess_file(file):
    records_dict_list = []
    with open("{}/{}".format(WEATHER_DATA_DIR,file), 'r') as f:
        data = f.read()
    json_data = json.loads(data)
    for record in json_data:
        tmp_df = flatten_json(record)
        records_dict_list.append(tmp_df)
    # Create dataframe of all dict records
    return pd.DataFrame(records_dict_list)

In [153]:
weather_df = preprocess_file('weather_troy.json')
weather_df.dropna(axis=1,how="all",inplace=True)
weather_df = weather_df[["main_temp","main_pressure","main_humidity","wind_speed","weather_0_main","weather_0_description","dt","rain_3h","snow_3h","dt_iso"]]
column_mapping = {"main_temp":"Temp","main_pressure":"Pressure","main_humidity":"Humidity","wind_speed":"Wind_Speed","weather_0_main":"Condition","weather_0_description":"Description","rain_3h":"Rain_3h","snow_3h":"Snow_3h","dt_iso":"Date_Time_ISO","dt":"Date_Time_Unix"}
weather_df.rename(columns=column_mapping,inplace=True)

In [157]:
weather_df

,Temp,Pressure,Humidity,Wind_Speed,Condition,Description,Date_Time_Unix,Rain_3h,Snow_3h,Date_Time_ISO
0,266.94,1020,67,8,Clouds,few clouds,1356998400,NaN,NaN,2013-01-01 00:00:00 +0000 UTC
1,266.68,1021,67,6,Clouds,few clouds,1357002000,NaN,NaN,2013-01-01 01:00:00 +0000 UTC
2,266.56,1021,62,6,Clouds,few clouds,1357005600,NaN,NaN,2013-01-01 02:00:00 +0000 UTC
3,266.69,1021,62,3,Clouds,broken clouds,1357009200,NaN,NaN,2013-01-01 03:00:00 +0000 UTC
4,266.82,1021,67,0,Clouds,broken clouds,1357012800,NaN,NaN,2013-01-01 04:00:00 +0000 UTC
...,...,...,...,...,...,...,...,...,...,...
52578,276.41,1019,80,2,Clouds,overcast clouds,1546286400,NaN,NaN,2018-12-31 20:00:00 +0000 UTC
52579,276.51,1021,72,4,Clouds,broken clouds,1546290000,NaN,NaN,2018-12-31 21:00:00 +0000 UTC
52580,276.51,1021,72,4,Clouds,overcast clouds,1546293600,NaN,NaN,2018-12-31 22:00:00 +0000 UTC
52581,276.08,1021,78,3,Rain,light rain,1546297200,NaN,NaN,2018-12-31 23:00:00 +0000 UTC


## Merge Weather and Crime Data

In [161]:
crime_weather_df = pd.merge(crime_df, weather_df, on='Date_Time_Unix', how='inner')
crime_weather_df.index.name = 'ID'

In [186]:
crime_weather_df['Date_Time_ISO'].dtype

dtype('O')

## Write to CSV file

In [163]:
crime_weather_df.to_csv(r'../crime_weather_data.csv')

In [187]:
crime_weather_df

,Incident_ID,Crime,Location,Date_Time,Longitude,Latitude,Agency,Accuracy,Address,Date_Time_Unix,Temp,Pressure,Humidity,Wind_Speed,Condition,Description,Rain_3h,Snow_3h,Date_Time_ISO
ID,,,,,,,,,,,,,,,,,,,
0,13-010407,Larceny,Residential,"Mar 28, 2013 3:33 PM",-73.6848,42.7360,Troy Police Department,Address,22XX OLD 6TH AVE,1364497200,282.21,1010,44,4,Clouds,few clouds,NaN,NaN,2013-03-28 19:00:00 +0000 UTC
1,13-010265,Burglary,Residential,"Mar 27, 2013 3:00 PM",-73.6913,42.7248,Troy Police Department,Address,1XX 3RD ST,1364410800,280.08,1007,37,2,Clear,sky is clear,NaN,NaN,2013-03-27 19:00:00 +0000 UTC
2,13-010245,Larceny,Residential,"Mar 27, 2013 12:10 PM",-73.6846,42.7125,Troy Police Department,Address,1XX DELAWARE AVE,1364400000,278.08,1007,0,1,Clouds,few clouds,NaN,NaN,2013-03-27 16:00:00 +0000 UTC
3,13-010143,Burglary,Residential,"Mar 26, 2013 3:30 PM",-73.6913,42.7250,Troy Police Department,Address,1XX 3RD ST,1364324400,279.34,1000,51,4,Clouds,overcast clouds,NaN,NaN,2013-03-26 19:00:00 +0000 UTC
4,13-010311,Larceny,School,"Mar 26, 2013 9:50 AM",-73.6706,42.7291,Troy Police Department,Address,19XX BURDETT AVE,1364302800,273.54,1006,0,3,Clouds,overcast clouds,NaN,NaN,2013-03-26 13:00:00 +0000 UTC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3805,18-005479,Agg. Assault,Street,"Jul 10, 2018 4:00 AM",-73.6887,42.7210,Troy Police Department,Intersection,6TH ST & JEFFERSON AVE,1531209600,288.14,1015,72,1,Mist,mist,NaN,NaN,2018-07-10 08:00:00 +0000 UTC
3806,18-005357,Robbery,Other Outside,"Jul 04, 2018 10:16 PM",-73.6904,42.7297,Troy Police Department,Address,XX 3RD ST,1530756000,300.48,1025,62,5,Clouds,few clouds,NaN,NaN,2018-07-05 02:00:00 +0000 UTC
3807,18-005356,Larceny,Residential,"Jul 04, 2018 1:45 PM",-73.6697,42.7248,Troy Police Department,Address,16XX TIBBITS AVE,1530723600,305.11,1026,46,1,Clouds,few clouds,NaN,NaN,2018-07-04 17:00:00 +0000 UTC
